In [49]:
import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix
import math
import pickle

In [50]:
data = pd.read_csv('./../data/round.csv')

In [51]:
data.shape

(254628, 12)

In [52]:
data.head()

,Tournament_Year,Tournament_#,Permanent_Tournament_#,Course_#,Player_Number,Player_Name,Round_Number,Tee_Time,Round_Score,End_of_Event_Pos._(text),Finishing_Pos,Finishing_Pct
0,2003,10,16,656,1320,"Estes, Bob",1,11:50,66,T6,7.5,0.197917
1,2003,10,16,656,1361,"Forsman, Dan",1,11:00,69,T21,22.0,0.600694
2,2003,10,16,656,1761,"Mattiace, Len",1,13:00,75,24,24.0,0.656250
3,2003,10,16,656,1797,"Mediate, Rocco",1,12:10,72,T2,2.5,0.059028
4,2003,10,16,656,1920,"Parry, Craig",1,11:20,71,T25,26.5,0.725694


In [53]:
with open('./../PickleFiles/num_to_ind_round.pkl','r') as pickleFile:
    num_to_inds = pickle.load(pickleFile)

In [54]:
data.insert(5,'Player_Index',[num_to_inds[num] for num in data.Player_Number])

In [55]:
rounds = data.groupby(['Tournament_Year','Permanent_Tournament_#','Round_Number','Course_#'],sort=False)

In [56]:
n_players = len(pd.unique(data.Player_Index))
n_rounds = len(rounds)
n_tournaments = len(data.groupby(['Tournament_Year','Permanent_Tournament_#']))

In [57]:
def save_sparse_csc(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

In [58]:
bin_size = 4
n_tournament_groups = int(math.ceil(n_tournaments/float(bin_size)))
current_group = 0
A,G = np.zeros((n_players,n_players)),np.zeros((n_players,n_players))
tournaments = set()
rounds_to_inflate = []
for (year,tourn,round,course),df in rounds:
    tournaments.add((year,tourn))
    tournament_group = (len(tournaments)-1)/bin_size
    if tournament_group>current_group:
        np.fill_diagonal(A,0)
        np.fill_diagonal(G,0)
        A = csc_matrix(A)
        G = csc_matrix(G)
        save_sparse_csc('./../rounds/%dA' % (current_group),A)
        save_sparse_csc('./../rounds/%dG' % (current_group),G)
        A,G = np.zeros((n_players,n_players)),np.zeros((n_players,n_players))
        current_group += 1
    
    A[np.ix_(pd.unique(df.Player_Index),pd.unique(df.Player_Index))] += \
            np.outer(1/df.Round_Score.values.astype(float),df.Round_Score.values)
    G[np.ix_(pd.unique(df.Player_Index),pd.unique(df.Player_Index))] += 1.

np.fill_diagonal(A,0)
np.fill_diagonal(G,0)
A = csc_matrix(A)
G = csc_matrix(G)
save_sparse_csc('./../rounds/%dA' % (current_group+1),A)
save_sparse_csc('./../rounds/%dG' % (current_group+1),G)